In [11]:
import openpyxl

def update_excel_file(tanggal, hari, jam, kelompok, operator, NIP):
    # Load the workbook and select the active worksheet
    workbook = openpyxl.load_workbook('QC Seiscomp.xlsx')
    sheet = workbook.active

    # Fill cell G2 with the specified value
    sheet['G2'] = ': ' + tanggal
    sheet['G3'] = ': ' + hari
    sheet['G4'] = f': {jam} - selesai'
    sheet['G5'] = f': {kelompok}'
    sheet['M18'] = tanggal
    sheet['M24'] = operator
    NIP = sheet['M25'] = 'NIP. ' + NIP

    # Save the workbook
    workbook.save('QC Seiscomp.xlsx')

# Example usage
update_excel_file('20 Desember 2020', 'Senin', '14:00', 'Kel. 1', 'Fajar', '199411052014111001')

In [7]:
import pandas as pd
from io import StringIO
from openpyxl.utils.dataframe import dataframe_to_rows
# Sample CSV string
csv_string = "apple,banana,cherry,orange\n1,2,3,4\n5,6,7,8"

# Create a DataFrame from the CSV string
df = pd.read_csv(StringIO(csv_string))
rows = dataframe_to_rows(df, index=False, header=False)

# print df rows size
print(len(df))
df['prev'] = 'prev'
df['no'] = range(1, len(df)+1)
print(df.shape[1])

for r_idx, row in enumerate(rows, 1):
    print(r_idx, row)

2
6
1 [1, 2, 3, 4, 'prev', 1]
2 [5, 6, 7, 8, 'prev', 2]


In [2]:
import subprocess
import os

def convert_xlsx_to_pdf(filename):
  """
  Converts an XLSX file to PDF using the LibreOffice command-line interface.

  Args:
    filename: Path to the input XLSX file.

  Returns:
    True if the conversion was successful, False otherwise.
  """
  try:
    command = ["soffice", "--headless", "--convert-to", "pdf:\"impress_pdf_Export\"", filename]
    subprocess.run(command, check=True) 
    return True
  except subprocess.CalledProcessError as e:
    print(f"Error converting {filename} to PDF: {e}")
    return False

# Example usage
filename = os.path.join(os.getcwd(), 'QC.xlsx')
print(filename)
# convert_xlsx_to_pdf(filename)

d:\Projects\QC\ebast\qc\static\qc\QC.xlsx


In [4]:
import requests
import pandas as pd

def clean_fm_data(data, start_datetime='2025-03-14 00:00:00', end_datetime='2025-03-15 00:00:00'):
    text = data.decode('utf-8')

    lines = text.split('\n')
    processed_lines = []
    for i, line in enumerate(lines):
        line = '|'.join(part.strip() for part in line.split('|'))
        processed_lines.append(line)

    df = pd.DataFrame([x.split('|') for x in processed_lines[1:]], columns=processed_lines[0].split('|'))
    df['Datetime (UTC)'] = pd.to_datetime(df['Datetime (UTC)'], format='%Y-%m-%d %H:%M:%S')

    def select_data_by_datetime_range(df, start_datetime, end_datetime):
        mask = (df['Datetime (UTC)'] >= start_datetime) & (df['Datetime (UTC)'] <= end_datetime)
        return df.loc[mask]

    df_selected = select_data_by_datetime_range(df, start_datetime, end_datetime)

    # sort the df_selected by 'Datetime (UTC)'
    df_selected = df_selected.sort_values(by='Datetime (UTC)')

    # divide the 'Datetime (UTC)' column into 'Date' and 'OT (UTC)' columns, and put them in the first two columns, remove the 'Datetime (UTC)' column
    df_selected['Date'] = df_selected['Datetime (UTC)'].dt.date
    df_selected['OT (UTC)'] = df_selected['Datetime (UTC)'].dt.time
    df_selected = df_selected[['Date', 'OT (UTC)'] + [col for col in df_selected.columns if col != 'Datetime (UTC)']]
    df_selected = df_selected.reset_index(drop=True)

    # sort the columns to be 'Date', 'OT (UTC)', 'Lat', 'Long', 'Mag', 'D(Km)', 'Phase', 'RMS', 'Az.Gap', 'Region', but first turn the respective column names into the desired ones
    df_selected = df_selected.rename(columns={'D': 'D(Km)', 'Type M': 'TypeMag'})
    df_selected = df_selected[['Date', 'OT (UTC)', 'Lat', 'Long', 'Mag', 'TypeMag', 'D(Km)', 'S1', 'D1', 'R1', 'S2', 'D2', 'R2', 'Fit(%)', 'CLVD (%)']]
    df_selected = df_selected.reset_index(drop=True)

    # Check for duplicate columns
    df_selected = df_selected.loc[:, ~df_selected.columns.duplicated()]
    
    return df_selected

def fetch_data(start_datetime='2025-03-14 00:00:00', end_datetime='2025-03-15 00:00:00'):
    url = "http://202.90.198.41/qc_focal.txt"
    response = requests.get(url)

    if response.status_code == 200:
        data = clean_fm_data(response.content, start_datetime, end_datetime)
        csv_data = data.to_csv(index=False)
        table_data = data.insert(0, 'No', range(1, len(data) + 1))
        table_data = data.to_dict(orient='records')
        return table_data
    #     return JsonResponse({'csv': csv_data, 'table_data': table_data})
    # else:
    #     return JsonResponse({'error': 'Failed to fetch data'}, status=500)
    
fetch_data()

[{'No': 1,
  'Date': datetime.date(2025, 3, 14),
  'OT (UTC)': datetime.time(3, 7, 11),
  'Lat': '-7.9950',
  'Long': '117.7203',
  'Mag': '4.3',
  'TypeMag': 'Mw',
  'D(Km)': '60',
  'S1': '135',
  'D1': '54',
  'R1': '113',
  'S2': '279',
  'D2': '42',
  'R2': '62',
  'Fit(%)': '71.13',
  'CLVD (%)': '1.06'},
 {'No': 2,
  'Date': datetime.date(2025, 3, 14),
  'OT (UTC)': datetime.time(9, 52, 13),
  'Lat': '-3.3746',
  'Long': '140.6872',
  'Mag': '4.6',
  'TypeMag': 'Mw',
  'D(Km)': '43',
  'S1': '146',
  'D1': '87',
  'R1': '151',
  'S2': '238',
  'D2': '61',
  'R2': '4',
  'Fit(%)': '82.02',
  'CLVD (%)': '1.10'},
 {'No': 3,
  'Date': datetime.date(2025, 3, 14),
  'OT (UTC)': datetime.time(11, 13, 32),
  'Lat': '1.1879',
  'Long': '120.2900',
  'Mag': '5.4',
  'TypeMag': 'Mw',
  'D(Km)': '35',
  'S1': '61',
  'D1': '24',
  'R1': '63',
  'S2': '270',
  'D2': '69',
  'R2': '101',
  'Fit(%)': '91.05',
  'CLVD (%)': '38.03'},
 {'No': 4,
  'Date': datetime.date(2025, 3, 14),
  'OT (UTC)